<h3 style='color:green;'>Encryption and Decryption using cryptography module</h3>

In [1]:
# installing the cryptography package
!pip install cryptography

In [2]:
# Verifying that the package has been installed
!pip show cryptography

Name: cryptography
Version: 44.0.1
Summary: cryptography is a package which provides cryptographic recipes and primitives to Python developers.
Home-page: https://github.com/pyca/cryptography
Author: The cryptography developers <cryptography-dev@python.org>
Author-email: The Python Cryptographic Authority and individual contributors <cryptography-dev@python.org>
License: Apache-2.0 OR BSD-3-Clause
Location: C:\Users\oroni\miniconda3\Lib\site-packages
Requires: cffi
Required-by: conda-content-trust


In [9]:
# importing the package to the Jupyter notebook
import cryptography

# importing the Fernet class from the fernet module in the Cryptograghy package to  generate the cryptographic keys
from cryptography.fernet import Fernet, MultiFernet

<h4>Data encryption</h4>

In [4]:
# Generating the key
key = Fernet.generate_key()
f = Fernet(key)

In [5]:
# The data(message) to be encrypted should be in bytes and not strings
message = b'Keen Ombasa'

In [6]:
fernettoken = f.encrypt(message)

# the encrypted message is stored as fernettoken.
fernettoken

b'gAAAAABoKITjnEe5UY6x-OLArO93PVTti0inJcMVixBJyGGFdR9ZRuWCVTSzJcTwkjKZKRg1c5Z63CAlXcxLfzgXjwqOxf1nbQ=='

<h4>Data decryption</h4>

In [7]:
decryptedmessage = f.decrypt(fernettoken)
decryptedmessage

b'Keen Ombasa'

In [8]:
# Converting the decrypted message from bytes to string we decode using UnicodeStandard
decryptedmessage.decode('utf-8')

'Keen Ombasa'

<h4>Using MultiFernet in cryptography</h4>

In [17]:
# Generating the keys
key1 = Fernet(Fernet.generate_key())
key2 = Fernet(Fernet.generate_key())
multifier = MultiFernet([key1, key2])

In [15]:
message = b'This is Secret Agent Keen Ombasa.'

In [18]:
# Encryting the message
fernettoken = multifier.encrypt(message)

In [19]:
print(fernettoken)

b'gAAAAABoKIz-BZzuC-k_G2ujpoYpXFiTmazVYoTwpCaYRc_0BTYf8UcEvjR2WY3WjuDcROoXCnKsKiTvN1H4-nb04yGZ5kSmbVIHcTHqNJYPWIZ8f9keDb62x-ZtE137d35hZRrJPMAN'


In [21]:
# Decrypting the message
multifier.decrypt(fernettoken).decode('utf-8')

'This is Secret Agent Keen Ombasa.'

<h4>Using passwords with Fernet</h4>

In [30]:
# 📦 1. Importing Required Modules
# Used to interact with the operating system, here to generate a random salt.
import os

# Used to encode binary data in ASCII format, especially to make keys URL-safe.
import base64

# Provides cryptographic hash functions (here SHA256).
from cryptography.hazmat.primitives import hashes

# A Key Derivation Function (KDF) based on PBKDF2 (Password-Based Key Derivation Function 2) using HMAC.
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

# Provides the default cryptographic backend for performing operations
from cryptography.hazmat.backends import default_backend

# 🔐 2. Defining Password and Salt
#  A byte string (b'...') representing the password. It must be in bytes for the key derivation function.
password = b'123456'

# Generates a 16-byte random salt, used to make the key derivation unique and secure
salt = os.urandom(16)
print(salt)

# 🧪 3. Creating the Key Derivation Function (KDF)
kdf = PBKDF2HMAC( 
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)

# 🔑 4. Deriving and Encoding the Key
key = base64.urlsafe_b64encode(kdf.derive(password))

# 🔐 5. Creating a Fernet Encryption Object
f = Fernet(key)

# ✉️ 6. Encrypting a Message
message = b'Keen Oroni Ombasa'
fernettoken = f.encrypt(message)
fernettoken


b'>+\xa3L\x04\x83\x92n\xcf\xd8r\x90?\xd2\xb3g'


b'gAAAAABoKJ8JO667Qd0FDtebm9akazlG9DAkXpMMQj0bBJqnoUSMY29F1GTVmdRAzhzY8S_HPams9rfrmA2DKJMSjXNF_RnvpDDr-UD8exg6r03a5d_UUbE='

In [24]:
f.decrypt(fernettoken).decode('utf-8')

'Keen Oroni Ombasa'

In [33]:
import os
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend

# === ENCRYPTION PHASE ===
def encrypt_message(password: str, message: bytes):
    password_bytes = password.encode()  # Convert password to bytes
    salt = os.urandom(16)  # Random salt (must be saved for decryption)

    # Derive key
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(password_bytes))

    # Encrypt
    fernet = Fernet(key)
    encrypted_message = fernet.encrypt(message)

    return encrypted_message, salt  # Return both encrypted message and salt


# === DECRYPTION PHASE ===
def decrypt_message(password: str, encrypted_message: bytes, salt: bytes):
    password_bytes = password.encode()

    # Derive key again using the same salt
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(password_bytes))

    # Decrypt
    fernet = Fernet(key)
    try:
        decrypted_message = fernet.decrypt(encrypted_message)
        return decrypted_message
    except Exception as e:
        return f"Decryption failed: {e}"


# === DEMO ===

# Step 1: User sets a password and a message
password_input = '123456'
message = b"Keen Oroni Ombasa"

# Encrypt
token, salt = encrypt_message(password_input, message)
print(f"\nEncrypted Message (token): {token}")
print(f"Salt (save this!): {salt}")

# Step 2: Later... user enters the same password to decrypt
password_attempt = input("\nEnter the password to decrypt: ")
decrypted = decrypt_message(password_attempt, token, salt).decode('utf-8')
print(f"\nDecrypted Message: {decrypted}")



Encrypted Message (token): b'gAAAAABoKKQszskC7c7KXLnqzZ8biM-beyTnMI_19jZNnxARlG00MS6wdbA_dxkXELeUlUumQWND4QKtZsCdF4TCkXtMpiIjabWEwIADrpnFcNHTokKEZ8M='
Salt (save this!): b'\xda<\xfc\x0b\xc4+\x89\xd4\xd4V*<)\xb7}\xbb'



Enter the password to decrypt:  123456



Decrypted Message: Keen Oroni Ombasa
